# Pozyskanie danych - czytania podczas mszy

In [1]:
import pandas as pd

## Język polski 
Pobieranie czytań ze strony: http://mateusz.pl/czytania

In [2]:
import requests
from bs4 import BeautifulSoup

def get_addresses(n):
    addresses = []
    date_list = [base - datetime.timedelta(days=x) for x in range(0, n)]
    dates_cleaned = [str(x).replace('-','') for x in date_list]
    for date in dates_cleaned:
        ymd = date
        address = "http://mateusz.pl/czytania/" + ymd[:4]+ "/" + ymd + ".html"
        addresses.append(address)
    return addresses

def get_readings(address, day_name):
    page = requests.get(address)
    soup = BeautifulSoup(page.content, 'html.parser')
    #print(soup.prettify()) 
    #Exception handling for more readings
    try:
        #section = list(soup.children)[9]
        #section_content = section.find_all('p')   
        section = list(soup.children)
        #if len(section) != 30:
            #print(len(section), address, day_name)
        section_content1 = section[2].find_all('p')
        if ((day_name == 'Sunday') or (len(section) in [32,34])):
            section_content2 = section[6]
            section_content3 = section[8]
            section_content4 = section[12]
        else:
            section_content2 = None
            section_content3 = section[4]
            section_content4 = section[8]
            
        pierwsze_czytanie = str(section_content1[6])
        drugie_czytanie = str(section_content2)
        tekst_przed_ewangelia = str(section_content3)
        psalm_ref = str(section_content1[7])
        psalm = ""
        i = 8
        while i < len(section_content1):
            psalm += str(section_content1[i])
            i += 1
        ewangelia = str(section_content4)
        document = [pierwsze_czytanie,psalm_ref,psalm, drugie_czytanie, tekst_przed_ewangelia, ewangelia]
        return document
    except:
        pass
        #section = soup.find_all('section')[0]
        #section_content =section.find_all('p')

In [3]:
import datetime

#czytanie na dziś
now = datetime.datetime.now()
today = datetime.date.today()
year = str(now.year)
month = str(now.month)
if len(month) == 1:
    month = "0" + month

day = str(now.day)
if len(day) == 1:
    day = "0" + day

ymd = year+month+day
today_name = now.strftime("%A")
day_name = today_name
#today_name = 'Sunday'

address = "http://mateusz.pl/czytania/" + year + "/" + ymd + ".html"
#address = "http://mateusz.pl/czytania/2018/20181021.html" #niedziela
doc = get_readings(address, today)
#print(address)

In [4]:
df_raw = pd.DataFrame(data=None, index=None, #pd.date_range('1/1/2019', periods=10, freq='D'), 
                  columns = ["Dzień tygodnia","Pierwsze czytanie", "Psalm_ref", "Psalm", "Drugie czytanie", 
                                                    "Werset przed Ewangelią", "Ewangelia"])
df_raw.head()

,Dzień tygodnia,Pierwsze czytanie,Psalm_ref,Psalm,Drugie czytanie,Werset przed Ewangelią,Ewangelia


In [7]:
n=7
base = datetime.date.today()
date_list = [base - datetime.timedelta(days=x) for x in range(0, n)]
dates_cleaned = [str(x).replace('-','') for x in date_list]
dates_names = [x.strftime("%A") for x in date_list]
addresses = get_addresses(n)

for i in range(n):
        doc = get_readings(addresses[i], dates_names[i])
        #doc_cleaned = [clean_text(str(text)) for text in doc]
        input_row = [dates_names[i]]
        for each in doc: #doc_cleaned:
            input_row.append(each)
        df_raw.loc[date_list[i]] = input_row
df_raw.head()

,Dzień tygodnia,Pierwsze czytanie,Psalm_ref,Psalm,Drugie czytanie,Werset przed Ewangelią,Ewangelia
2019-07-07,Sunday,"<p>\r\n (Iz 66,10-14c) ...","<p>\r\n (Ps 66,1-3.4-7....","<p>Z radością sławcie Boga, wszystkie ziemie,<...","<p>\r\n (Ga 6,14-18) <b...",<p>\r\n Aklamacja (Kol ...,"<p>\r\n (Łk 10,1-12.17-..."
2019-07-06,Saturday,"<p>\r\n (Rdz 27,1-5.15-...","<p>\r\n (Ps 135,1-6) <b...","<p>Chwalcie imię Pana,<br/>\r\nchwalcie Pańscy...",None,<p>\r\n Aklamacja (Jk 1...,"<p>\r\n (Mt 9,14-17) <b..."
2019-07-05,Friday,"<p>\r\n (Rdz 23,1-4.19....","<p>\r\n (Ps 106,1-5) <b...","<p>Chwalcie Pana, bo jest dobry,<br/>\r\nbo na...",None,<p>\r\n Aklamacja (Mt 9...,"<p>\r\n (Mt 9,9-13) <br..."
2019-07-04,Thursday,"<p>\r\n (Rdz 22,1-19) <...","<p>\r\n (Ps 116,1-6.8-9...","<p>Miłuję Pana, albowiem usłyszał<br/>\r\ngłos...",None,<p>\r\n Aklamacja (2 Ko...,"<p>\r\n (Mt 9,1-8) <br>..."
2019-07-03,Wednesday,"<p>\r\n (Ef 2,19-22) <b...","<p>\r\n (Ps 117,1-2) <b...","<p>Chwalcie Pana, wszystkie narody,<br/>\r\nwy...",None,<p>\r\n Aklamacja (J 20...,"<p>\r\n (J 20,24-29) <b..."


### Czyszczenie do formy tekstu zrozumiałego dla człowieka

In [8]:
import re

#first phase - for easy reading
def clean_html(raw):
    cleaner = re.compile('<.*?>')
    cleantext = re.sub(cleaner, '', raw)
    return cleantext

def clean_proverbs (raw):
    cleaner = re.compile('\([^)]*\)')
    cleantext = re.sub(cleaner, '',raw)
    return cleantext

def clean_indent(raw):
    cleaner = re.compile('^\s+')
    cleantext = re.sub(cleaner, '',raw)
    return cleantext

def clean_rest(text):
    cleaner = re.compile('\\r\\n\s+')
    cleantext = re.sub(cleaner, '',text)
    return cleantext

def clean_rest2(text):
    cleaner = re.compile('(\\r\\n)')
    cleantext = re.sub(cleaner, ' ',text)
    return cleantext

def clean_carriage(text):
    cleaner = re.compile('(.*)\n')
    cleantext = re.sub(cleaner, '',text)
    return cleantext

def clean_psalmus(text):
    cleaner = re.compile('Aklamacja|REFREN\:\s|Bracia\:\s')
    cleantext = re.sub(cleaner, '',text)
    return cleantext

def clean_text(raw):
    text = clean_html(raw)
    text = clean_proverbs(text)
    text = clean_indent(text)
    text = clean_rest(text)
    text = clean_rest2(text)
    text = clean_carriage(text)
    text = clean_psalmus(text)
    return text

In [9]:
df = df_raw.copy()
for each in df_raw.columns:
    df[each] = df[each].apply(lambda x: clean_text(x))
df.head(7)

,Dzień tygodnia,Pierwsze czytanie,Psalm_ref,Psalm,Drugie czytanie,Werset przed Ewangelią,Ewangelia
2019-07-07,Sunday,"Radujcie się wraz z Jerozolimą, weselcie się w...",Niech cała ziemia chwali swego Boga,"Z radością sławcie Boga, wszystkie ziemie, opi...","Nie daj Boże, bym się miał chlubić z czego inn...",Sercami waszymi niech rządzi Chrystusowy pok...,Jezus wyznaczył jeszcze innych siedemdziesięci...
2019-07-06,Saturday,Gdy Izaak się zestarzał i jego oczy stały się ...,"Śpiewajcie Panu, bo nasz Pan jest dobry","Chwalcie imię Pana, chwalcie Pańscy słudzy, wy...",None,"Ze swej woli zrodził nas przez słowo prawdy,...",Uczniowie Jana podeszli do Jezusa i zapytali: ...
2019-07-05,Friday,Sara doczekała się stu dwudziestu siedmiu lat ...,Nasz Pan jest dobry: chwalcie Go na wieki,"Chwalcie Pana, bo jest dobry, bo na wieki trwa...",None,"Nie potrzebują lekarza zdrowi, lecz ci, któr...",Odchodząc z Kafarnaum Jezus ujrzał człowieka s...
2019-07-04,Thursday,Bóg wystawił Abrahama na próbę. Rzekł do niego...,W krainie życia będę widział Boga,"Miłuję Pana, albowiem usłyszał głos mego błaga...",None,"W Chrystusie Bóg pojednał świat ze sobą, nam...","Jezus wsiadł do łodzi, przeprawił się z powrot..."
2019-07-03,Wednesday,"Nie jesteście już obcymi i przychodniami, ale ...",Idźcie i głoście światu Ewangelię,"Chwalcie Pana, wszystkie narody, wysławiajcie ...",None,"Uwierzyłeś Tomaszu, bo Mnie ujrzałeś. Błogos...","Tomasz, jeden z Dwunastu, zwany Didymos, nie b..."
2019-07-02,Tuesday,"Aniołowie przynaglali Lota, mówiąc: „Prędzej, ...",Twoją łaskawość zawsze mam w pamięci,"Doświadcz mnie, Panie, wystaw mnie na próbę, z...",None,"W Panu pokładam nadzieję, nadzieję żywi moja...","Gdy Jezus wszedł do łodzi, poszli za Nim Jego ..."
2019-07-01,Monday,"Mężowie, których gościł Abraham, odeszli i ski...","Pan jest łaskawy, pełen miłosierdzia","Błogosław, duszo moja, Pana, i wszystko, co je...",None,"Nie zatwardzajcie dzisiaj serc waszych, lecz...","Gdy Jezus zobaczył tłum dokoła siebie, kazał o..."


### Przykład

In [13]:
df.set_index(pd.Index([str(x) for x in df.index]), inplace=True) # zamiania indeksu z formatu datetime() na str()
df.iloc[0]['Ewangelia']

'Jezus wyznaczył jeszcze innych siedemdziesięciu dwóch uczniów i wysłał ich po dwóch przed sobą do każdego miasta i miejscowości, dokąd sam przyjść zamierzał. Powiedział też do nich: ”Żniwo wprawdzie wielkie, ale robotników mało; proście więc Pana żniwa, żeby wyprawił robotników na swoje żniwo. Idźcie, oto was posyłam jak owce między wilki. Nie noście z sobą trzosa ani torby, ani sandałów; i nikogo w drodze nie pozdrawiajcie. Gdy do jakiego domu wejdziecie, najpierw mówcie: "Pokój temu domowi". Jeśli tam mieszka człowiek godny pokoju, wasz pokój spocznie na nim; jeśli nie, powróci do was. W tym samym domu zostańcie, jedząc i pijąc, co mają; bo zasługuje robotnik na swą zapłatę. Nie przechodźcie z domu do domu. Jeśli do jakiego miasta wejdziecie i przyjmą was, jedzcie, co Wam podadzą; uzdrawiajcie chorych, którzy tam są, i mówcie im: "Przybliżyło się do was królestwo Boże". Lecz jeśli do jakiego miasta wejdziecie, a nie przyjmą was, wyjdźcie na jego ulice i powiedzcie: "Nawet proch, któ

### Zapis do pliku csv

In [14]:
df.to_csv(path_or_buf= '../CSV/czytania.csv', header=df.columns, sep=',')

## Język angielski
Pobieranie czytań ze strony: http://www.usccb.org/bible/readings/

In [15]:
address =  "http://www.usccb.org/bible/readings/"
print(address)

http://www.usccb.org/bible/readings/


In [16]:
def get_addresses(n):
    addresses = []
    date_list = [base - datetime.timedelta(days=x) for x in range(0, n)]
    dates_cleaned = [str(x).replace('-','') for x in date_list]
    for date in dates_cleaned:
        ymd = date
        address = "http://www.usccb.org/bible/readings/" + ymd[4:6] + ymd[-2:] + ymd[2:4] + ".cfm"
        addresses.append(address)
    return addresses

def get_readings(address):
    try:
        page = requests.get(address)
        soup = BeautifulSoup(page.content, 'html.parser')
        section = list(soup.children)[8]
        section = list(section.children) [15]
        first = section.find_all('div')[11]
        psalm = section.find_all('div')[15]
        vers = section.find_all('div')[19]
        gospel = section.find_all('div')[23]
        document = [first, psalm, vers, gospel]
        return document
    except:
        pass

In [17]:
df_raw = pd.DataFrame(data=None, index=None, #pd.date_range('1/1/2019', periods=10, freq='D'), 
                  columns = ["Dzień tygodnia","Pierwsze czytanie","Psalm","Werset przed Ewangelią", "Ewangelia"])
df_raw.head()

,Dzień tygodnia,Pierwsze czytanie,Psalm,Werset przed Ewangelią,Ewangelia


In [18]:
n=7
base = datetime.date.today()
date_list = [base - datetime.timedelta(days=x) for x in range(0, n)]
dates_cleaned = [str(x).replace('-','') for x in date_list]
dates_names = [x.strftime("%A") for x in date_list]
addresses = get_addresses(n)

for i in range(n):
        doc = get_readings(addresses[i])
        input_row = [dates_names[i]]
        for each in doc: 
            input_row.append(each)
        df_raw.loc[date_list[i]] = input_row
df_raw.head(7)

,Dzień tygodnia,Pierwsze czytanie,Psalm,Werset przed Ewangelią,Ewangelia
2019-07-07,Sunday,"<div class=""poetry""> Thus says the LORD:<br/> ...","<div class=""poetry""> R. (1) <strong>Let all th...","<div class=""poetry""> Brothers and sisters:<br/...","<div class=""bibleReadingsWrapper""> <h4>Gospel ..."
2019-07-06,Saturday,"<div class=""poetry""> When Isaac was so old tha...","<div class=""poetry""> R.(3a) <strong>Praise the...","<div class=""poetry""> R. <strong>Alleluia, alle...","<div class=""poetry""> The disciples of John app..."
2019-07-05,Friday,"<div class=""poetry""> The span of Sarah's life ...","<div class=""poetry""> R. (1b) <strong>Give than...","<div class=""poetry""> R. <strong>Alleluia, alle...","<div class=""poetry"">As Jesus passed by,<br> he..."
2019-07-04,Thursday,"<div class=""poetry""> God put Abraham to the te...","<div class=""poetry""> R.(9) <strong>I will walk...","<div class=""poetry""> R. <strong>Alleluia, alle...","<div class=""poetry"">After entering a boat, Jes..."
2019-07-03,Wednesday,"<div class=""poetry""> Brothers and sisters:<br/...","<div class=""poetry""> R.(Mark 16:15) <strong>Go...","<div class=""poetry""> R. <strong>Alleluia, alle...","<div class=""poetry"">Thomas, called Didymus, on..."
2019-07-02,Tuesday,"<div class=""poetry""> As dawn was breaking, the...","<div class=""poetry""> R.(3a) <strong>O Lord, yo...","<div class=""poetry""> R. <strong>Alleluia, alle...","<div class=""poetry""> As Jesus got into a boat,..."
2019-07-01,Monday,"<div class=""poetry""> Abraham and the men who h...","<div class=""poetry""> R. (8a) <strong>The Lord...","<div class=""poetry"">R.<strong> Alleluia, allel...","<div class=""poetry""> When Jesus saw a crowd ar..."


In [193]:
#to do: cleaning